In [1]:
from urllib.request import urlopen
from fastai.vision import *
from fastai.metrics import error_rate
from PIL import Image
#from google.colab import drive
from os import listdir
import numpy as np 
import cv2
from matplotlib import image
from matplotlib import pyplot
import numpy as np
import warnings
import pandas as pd
import os 
import torch
from fastai.vision.data import *
import fastai
from pathlib import Path
from shutil import copyfile
import random
import shutil
from PIL import Image
from datetime import datetime
from hyperopt import hp
from hyperopt import fmin, tpe, space_eval
from hyperopt import  Trials
import torch.nn as nn
from datetime import datetime
from time import mktime
import time
from sklearn.model_selection import RepeatedKFold

In [2]:
def get_float_label(x):
    y = str(x)[(len(path_str) + 1):(len(path_str) + 3)]
    dash_search = re.search("/", y)
    if dash_search != None:
        y = y[:dash_search.span()[0]]
    y = float(y)
    return y

In [3]:
path_str = "/media/revilo/oliver-kool-2/pawpularity_data/train_for_resnet"
path = Path(path_str)

warnings.filterwarnings('ignore')

class L1LossFlat(nn.L1Loss):
#“Mean Absolute Error Loss”
  def forward(self, input:Tensor, target:Tensor) -> Rank0Tensor:
    return super().forward(input.view(-1), target.view(-1))


In [4]:

current_dir = ! pwd
current_dir = current_dir[0]

# Fastai trainable parameters for image processing and
# CNN

space = hp.choice('a',
    [('MSE loss', 
        hp.choice('size', [64, 128, 224]), # image size
        hp.uniform('weight_dec', 0, 0.99), # weight decay
         hp.uniform('div_factor', 1, 100), # div_factor
         hp.uniform('mom1',  0.5,0.85),  # momemtum1
         hp.uniform('mom2',  0.86, 0.99),  # momemtum2
         hp.uniform("max_rotate", 0, 360), # rotation angle
         hp.uniform("max_zoom", 1, 3), # max_zoom
         hp.uniform("p_affine", 0, 0.99), # p_affine,
         hp.uniform("max_lighting", 0, 0.99), # max_lighting
         hp.uniform("max_warp", 0, 0.99),# max warp 
        hp.choice("loss_fn", [MSELossFlat, L1LossFlat]),
        hp.choice("model", [models.resnet18, models.resnet34,
                           models.resnet50])
     )
    ])
                  

def rmse_basic(x, y):
    return np.sqrt(np.mean((x-y)**2))

## CV as part of objective function

In [5]:
# file names from train.sv

data_dir =  "/media/revilo/oliver-kool-2/pawpularity_data/"
df = pd.read_csv(f'{data_dir}/train.csv')
df.Id = df.Id.apply(lambda x: x + ".jpg")
df = df[["Id", "Pawpularity"]]#.rename(columns = {"Pawpularity":})
df["Id"] = df["Pawpularity"].astype(str) + "/" +  df["Id"]

In [14]:
#from sklearn.model_selection import RepeatedStratifiedKFold
def get_test_indices(df, no_of_folds = 5):
    #random_state = 12883823
    rkf = RepeatedKFold(n_splits=no_of_folds, 
                        n_repeats=1) #random_state=random_state
    test_indexes = []
    for train, test in rkf.split(df):
        test_indexes.append(test)
    return test_indexes

In [19]:
rmses_by_fold = []

def objective(args):
    (label,
     size,
    weight_dec, 
    div_factor,
    mom1, 
    mom2, 
    max_rotate,
    max_zoom,
    p_affine,
    max_lighting, 
    max_warp,
    loss_fn,
    model) = args
    
    test_indexes = get_test_indices(df, no_of_folds = 5)
    rmse_list = []
    
    for test_index_array in test_indexes[0:2]:
        
        data = (ImageList.from_folder(path)\
            .split_by_idx(test_index_array)\
            .label_from_func(get_float_label, label_cls = 
               FloatList)\
        .transform(get_transforms(do_flip=False,
                                 max_rotate = max_rotate,
                                 max_zoom = max_zoom,
                                 p_affine = p_affine,
                                 max_lighting = max_lighting,
                                 max_warp = max_warp), 
                   size=size)\
        .databunch())

        data.normalize(imagenet_stats) 

        learn = cnn_learner(data, model, model_dir = Path(current_dir))
        learn.loss = loss_fn
        learn.fit_one_cycle(3,
                            div_factor = div_factor,
                            moms = (mom1, mom2),
                            wd = weight_dec)

        interp = ClassificationInterpretation.from_learner(learn)
        rmse_res = rmse_basic(interp.preds.numpy().reshape(-1), 
            interp.y_true.numpy()) 
        rmse_list.append(rmse_res)
        
        t1 = time.localtime()
        dt = datetime.fromtimestamp(mktime(t1))
    
        learn.save(f'resnet_models/{str(dt)}_{np.round(rmse_res, 2)}.pt')
        del learn
    
    final_rmse = np.mean(rmse_list)
    rmses_by_fold.append(rmse_list)

    return final_rmse



In [20]:
print("Baseline RMSE to beat: 20.5851")
baseline_rmse = 20.5851

trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=10,
           trials = trials)

losses = [x["loss"] for x in trials.results]

if losses[len(losses) - 1] < baseline_rmse:
    print("RMSE below baseline")

print(best)
print(space_eval(space, best))


Baseline RMSE to beat: 20.5851
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

epoch,train_loss,valid_loss,time
0,1229.530518,740.440796,00:51
1,419.720917,325.706177,00:51
2,333.306122,321.607758,00:51


epoch,train_loss,valid_loss,time
0,1233.709717,791.319031,00:50
1,424.450409,332.683624,00:51
2,333.734741,329.429535,00:51


 10%|█         | 1/10 [05:32<49:50, 332.30s/trial, best loss: 18.04180908203125]

epoch,train_loss,valid_loss,time
0,1228.265381,685.556152,00:48


 10%|█         | 1/10 [06:26<57:57, 386.41s/trial, best loss: 18.04180908203125]


KeyboardInterrupt: 

In [38]:
trials_df = pd.DataFrame(trials.trials)
trials_df_small = pd.DataFrame(trials.vals)
trials_df["RMSE"] = trials.losses()
trials_df_small["RMSE"] = trials.losses()

t1 = time.localtime()
dt = datetime.fromtimestamp(mktime(t1))
trials_df.to_csv(f'resnet_trials/trials_df_{dt}.csv')
trials_df.to_csv(f'resnet_trials/trials_df_small_{dt}.csv')
